In [ ]:
import os
import geopandas as gp
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
from pylab import *
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
plt.rcParams['axes.unicode_minus']=False#负号
plt.rcParams['font.sans-serif']=['Arial']###############
plt.rcParams['font.size'] = '7' # 设置字体大小 = '16' # 设置字体大小
# plt.rcParams["font.weight"] = "bold"
# plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams['xtick.direction'] = 'in'#将x周的刻度线方向设置向内
plt.rcParams['ytick.direction'] = 'in'#将y轴的刻度方向设置向内

In [ ]:
# latlon_1degree_SJ_last=path+'/global_grid.shp'
version='v16_250104'
output_dir = os.path.join(r'J:\lakemapping\postprocess',version)
excel_dir=os.path.join(output_dir,f'stastics_excel')

gdb=os.path.join(output_dir,f'7_correct_temp_file.gdb')
fig_path=r'D:\lakemapping\10_paper_writting\figure\v4_250120'
os.makedirs(fig_path,exist_ok=True)

global_diss=r'D:\lakemapping\0_auxiliary_data\World_Continents\world_vector\World_continents_dissolve.shp'
Arid_Region=r'D:\lakemapping\0_auxiliary_data\AI\Global-AI_ET0_annual_v3\Global-AI_ET0_v3_annual\Global_AI_Reclassify_Clip_simple.shp'

color_list=['#B8DBB3','#72B063','#E29135','#94C6CD','#719AAC','#4A5F7E','#9e003a']#浅绿，深绿，橙色，浅蓝，中蓝，深蓝,红
label_list=['0.005~0.01 ${km^2}$','0.01~0.03 ${km^2}$','0.03~0.1 ${km^2}$','0.1~1 ${km^2}$','>1 ${km^2}$']
sub_fig_name=['a','b','c','d','e','f']
red_color_list=['#311d3f','#522546','#88304e']
# lake_category_bins=[0.005,0.01,1,100,10000,1000000]
# lake_category_labels=['Tiny','Small','Medium','Large','Super large']

lake_category_bins=[0.005,0.01,1,100,10000,1000000]
lake_category_labels=['1','3','10','100']

In [ ]:
latlon_1degree_SJ_last_data=gp.read_file(gdb,layer='Global_grid_shp_1degree_hu_pld')
latlon_1degree_SJ_last_data

In [ ]:
Lat_data = latlon_1degree_SJ_last_data.drop(columns='geometry').groupby(['Lat_down']).sum().reset_index()
latitude= Lat_data['Lat_down'].values
Lat_data

In [ ]:
Lon_data['hu_a1'].max()
Lon_data['hu_a_bt1_10'].max()

In [ ]:
Lon_data = latlon_1degree_SJ_last_data.drop(columns='geometry').groupby(['Lon_left']).sum().reset_index()
longitude= Lon_data['Lon_left'].values
Lon_data

In [ ]:
def add_background_fig(ax,ax_legend,data,terrain_map,column,legend_kwds,norm,grid_flag=True):
    map1 = Basemap( llcrnrlat=-60, urcrnrlat=90, llcrnrlon=-180, urcrnrlon=180,ax=ax)
    map1.readshapefile(global_diss[:-4],'global_diss',drawbounds=True,color='k',linewidth=0.4)
    
    tu=data.plot(
            ax=ax,
            cax=ax_legend,
            zorder=1,
            legend=True,  # 图例
            column=column,  # 分级设色字段#picover0\pia_0
            cmap=terrain_map,#terrain_map,  # 渐变色带的名称#Set2  #seismic
            legend_kwds=legend_kwds,
            norm=norm
            )
    
def draw_line(ax,xlim,xticks,ylim,yticks,line_color,label,x,y):
    ax.set_xlim(xlim)
    ax.set_xticks(xticks)
    ax.set_ylim(ylim)
    ax.set_yticks(yticks)
    ax.plot(x, y, color=line_color, linestyle = "-",linewidth = 0.8,label=label)
    
    
def custom_ticks(x, pos):
        if x == 0:
            return "0"
        poww=int(math.log10(x))
        return f"10$^{poww}$"

In [ ]:
savepic=os.path.join(fig_path,'3-3 compare with PLD.png')
fig=plt.figure(figsize=(6.5,6.25),dpi=300)#fig=plt.figure(figsize=(18,7.9))
gs = gridspec.GridSpec(4,3,width_ratios=[0.12,6,0.8],height_ratios=[0.8,2.5,2.5,0.8],wspace =0.25,hspace =0.18)

columns=['diff_a_bt1_10','diff_c_bt1_10']
grid_labels=[[0,0,1,0],[0,0,0,1]]
titles=['Difference of area for lake \n between 0.01 and 0.1(km$^2$)','Difference of count for lake \n between 0.01 and 0.1(km$^2$)']

RdYlBu_r = plt.colormaps.get_cmap('RdBu_r')#
newcolors = RdYlBu_r(np.linspace(0,1, 256))
terrain_map = ListedColormap(newcolors)
    
b1=40
b2=1200
norms=[colors.Normalize(vmin=-b1, vmax=b1),colors.Normalize(vmin=-b2, vmax=b2)]
ticks=[np.arange(-b1,b1+1,10),np.arange(-b2,b2+1,400)]

## 面积、数量纬度分布图
### a.经度面积
ax1=plt.subplot(gs[0, 1],)
ax1.text(-175,7.5,sub_fig_name[0],fontsize=12)
s=4
a_lt1=Lon_data['hu_a1'].values/(10.0**s)
a_bt1_10=Lon_data['hu_a_bt1_10'].values/(10.0**s)
a_gt10=Lon_data['hu_a0'].values/(10.0**s)-a_lt1-a_bt1_10
bottom=[0,a_gt10,a_gt10+a_bt1_10]
height=[a_gt10,a_bt1_10,a_lt1]

a_bt1_10=Lon_data['pld_a_bt1_10'].values/(10.0**s)
a_gt10=Lon_data['pld_a0'].values/(10.0**s)-a_bt1_10
bottom_p=[0,a_gt10]
height_p=[a_gt10,a_bt1_10]

alpha=[0.8,0.5,0.2]
label=['>0.1 km$^2$','0.01~0.1 km$^2$','0.005~0.01 km$^2$']
for i in range(0,3):
    ax1.bar(longitude+0.5,bottom=bottom[i],height=height[i], color='#9e003a',width=1,alpha=alpha[i],label=label[i])
for i in range(0,2):    
    ax1.bar(longitude+0.5,bottom=bottom_p[i],height=height_p[i],color='#719AAC',width=1,alpha=alpha[i],label=label[i])

ax1.text(-180,10,f'×10$^{s}$',ha='left',va='bottom')
ax1.set_ylabel('Area ($\mathregular{km^2}$)')
ax1.legend(loc=[1.02,0],frameon=False)
ax1.set_ylim(0,10)

### b.格网面积
ax22 = plt.subplot(gs[1, 0])
ax2=plt.subplot(gs[1, 1])
ax2.text(-175,75,sub_fig_name[1],fontsize=12)
for spine in ax2.spines.values():
    spine.set_visible(False)
legend_kwds={'orientation': "vertical",'extend':'both','ticks': ticks[0]}
add_background_fig(ax2,ax22,latlon_1degree_SJ_last_data,terrain_map,columns[0],legend_kwds,norms[0])
ax22.text(-1,0, titles[0],rotation=90,ha='right',va='center')

### c.纬度面积
ax3=plt.subplot(gs[1,2])
ax3.text(2,77,sub_fig_name[2],fontsize=12)

s=4
a_lt1=Lat_data['hu_a1'].values/(10.0**s)
a_bt1_10=Lat_data['hu_a_bt1_10'].values/(10.0**s)
a_gt10=Lat_data['hu_a0'].values/(10.0**s)-a_lt1-a_bt1_10
left=[0,a_gt10,a_gt10+a_bt1_10]
width=[a_gt10,a_bt1_10,a_lt1]
a_bt1_10=Lat_data['pld_a_bt1_10'].values/(10.0**s)
a_gt10=Lat_data['pld_a0'].values/(10.0**s)-a_bt1_10
left_p=[0,a_gt10]
width_p=[a_gt10,a_bt1_10]

for i in range(0,3):
    ax3.barh(latitude+0.5,left=left[i],width=width[i], color='#9e003a',height=1,alpha=alpha[i],label=label[i])
for i in range(0,2):    
    ax3.barh(latitude+0.5,left=left_p[i],width=width_p[i], color='#719AAC',height=1,alpha=alpha[i],label=label[i])

ax3.set_xlim(0,15)
ax3.text(15.5,-60,f'×10$^{s}$',ha='left',va='bottom')

### d.格网数量
ax44 = plt.subplot(gs[2, 0])
ax4=plt.subplot(gs[2, 1])
ax4.text(-175,75,sub_fig_name[3],fontsize=12)
for spine in ax4.spines.values():
        spine.set_visible(False)
legend_kwds={'orientation': "vertical",'extend':'both','ticks': ticks[1]}
add_background_fig(ax4,ax44,latlon_1degree_SJ_last_data,terrain_map,columns[1],legend_kwds,norms[1])
ax44.text(-1,0, titles[1],rotation=90,ha='right',va='center')

### e.纬度数量
ax5=plt.subplot(gs[2,2])
ax5.text(0.75,77,sub_fig_name[4],fontsize=12)
left=[0,c_gt10,c_gt10+c_bt1_10]
width=[c_gt10,c_bt1_10,c_lt1]
s=5
c_lt1=Lat_data['hu_c1'].values/(10.0**s)
c_bt1_10=Lat_data['hu_c_bt1_10'].values/(10.0**s)
c_gt10=Lat_data['hu_c0'].values/(10.0**s)-c_lt1-c_bt1_10
left=[0,c_gt10,c_gt10+c_bt1_10]
width=[c_gt10,c_bt1_10,c_lt1]
c_bt1_10=Lat_data['pld_c_bt1_10'].values/(10.0**s)
c_gt10=Lat_data['pld_c0'].values/(10.0**s)-c_bt1_10
c0=Lat_data['pld_c0'].values/(10.0**s)
left_p=[0,c_gt10]
width_p=[c_gt10,c_bt1_10]
line_width=[c0,c_bt1_10]
label_p=['PLD','PLD <0.1 km$^2$']
for i in range(0,3):
    ax5.barh(latitude+0.5,left=left[i],width=width[i],color='#9e003a',height=1,alpha=alpha[i],label=label[i])
for i in range(0,2):    
    ax5.barh(latitude+0.5,left=left_p[i],width=width_p[i], color='#719AAC',height=1,alpha=alpha[i],label=label[i])
#     ax5.plot(latitude+0.5,line_width[i], color='#719AAC',alpha=alpha[i],label=label_p[i])
    
ax5.text(6.2,-60,f'×10$^{s}$',ha='left',va='bottom')
ax5.set_xlim(0,6)

### f.经度数量
ax6=plt.subplot(gs[3, 1],)
ax6.text(-175,1.4,sub_fig_name[5],fontsize=12)

s=5
c_lt1=Lon_data['hu_c1'].values/(10.0**s)
c_bt1_10=Lon_data['hu_c_bt1_10'].values/(10.0**s)
c_gt10=Lon_data['hu_c0'].values/(10.0**s)-c_lt1-c_bt1_10
bottom=[0,c_gt10,c_gt10+c_bt1_10]
height=[c_gt10,c_bt1_10,c_lt1]

c_bt1_10=Lon_data['pld_c_bt1_10'].values/(10.0**s)
c_gt10=Lon_data['pld_c0'].values/(10.0**s)-c_bt1_10
bottom_p=[0,c_gt10]
height_p=[c_gt10,c_bt1_10]

for i in range(0,3):
    ax6.bar(longitude+0.5,bottom=bottom[i],height=height[i], color='#9e003a',width=1,alpha=alpha[i],label=label[i])
for i in range(0,2):    
    ax6.bar(longitude+0.5,bottom=bottom_p[i],height=height_p[i], color='#719AAC',width=1,alpha=alpha[i],label=label[i])
    
ax6.text(-180,2,f'×10$^{s}$',ha='left',va='bottom')
ax6.set_ylim(0,2)
ax6.set_ylabel('Count')
# ax6.legend(loc=[1.02,-0.1],frameon=False)

for ax in[ax1,ax6]:
    ax.set_xlim(-180,180)
    ax.set_xticks(range(-180,181,60))
    ax.set_xlabel('Longitude')
    
for ax in[ax3,ax5]:
    ax.set_ylim(-60,90)
    ax.set_yticks(range(-60,91,30))
    ax.set_ylabel('Latitude')
    ax.yaxis.set_label_position("right")
plt.savefig(savepic,dpi=400,bbox_inches='tight')#,dpi=400
plt.show()

In [ ]:
savepic=os.path.join(fig_path,'3-3 compare with PLD invert axis a.png')
fig=plt.figure(figsize=(6.5,1),dpi=300)#fig=plt.figure(figsize=(18,7.9))
gs = gridspec.GridSpec(2,1,height_ratios=[1,1],hspace =0)

### a.经度面积
ax1=plt.subplot(gs[0, 0],)
ax1.text(-180,3.5,sub_fig_name[0],fontsize=12)

s=3
hu_a_lt1=Lon_data['hu_a1'].values/(10.0**s)
hu_a_bt1_10=Lon_data['hu_a_bt1_10'].values/(10.0**s)
pld_a_bt1_10=Lon_data['pld_a_bt1_10'].values/(10.0**s)

alpha=[0.7,0.5,0.2]
label=['>0.1 km$^2$','0.01~0.1 km$^2$','0.005~0.01 km$^2$']

ax1.bar(longitude+0.5,bottom=hu_a_bt1_10,height=hu_a_lt1, color='#9e003a',width=1,alpha=alpha[2],label=label[2])
ax1.bar(longitude+0.5,hu_a_bt1_10, color='#9e003a',width=1,alpha=alpha[1],label=label[1])
ax1.bar(longitude+0.5,pld_a_bt1_10,color='#719AAC',width=1,alpha=alpha[1],label=label[1])

ax1.set_ylim(0,3)
ax1.set_xlim(-180,180)
ax1.text(-175,2,f'×10$^{s}$',ha='left',va='bottom')
ax1.set_ylabel('Area ($\mathregular{km^2}$)')
ax1.set_yticks(range(0,4))

ax11=plt.subplot(gs[1, 0],)
s=4
hu_a_gt10=Lon_data['hu_a_gt10'].values/(10.0**s)
pld_a_gt10=Lon_data['pld_a_gt10'].values/(10.0**s)
ax11.bar(longitude+0.5,-hu_a_gt10, color='#9e003a',width=1,alpha=alpha[0],label=label[0])
ax11.bar(longitude+0.5,-pld_a_gt10,color='#719AAC',width=1,alpha=alpha[0],label=label[0])
ax11.set_ylim(-9,0)
ax11.set_xlim(-180,180)

ax1.spines['top'].set_color('none')
ax1.spines['right'].set_color('none')
ax1.spines['left'].set_linewidth(0.5) 
ax1.spines['bottom'].set_linewidth(0.5) 
ax11.spines['top'].set_position('zero')   # x轴在顶部
ax11.spines['right'].set_position(('data', 180)) 
ax11.spines['right'].set_linewidth(0.5) 
ax11.spines['left'].set_color('none')  # 隐藏左侧轴
ax11.spines['bottom'].set_color('none')  # 隐藏底部轴
ax11.xaxis.set_ticks_position('top')  # 将x轴刻度移动到顶部
ax11.yaxis.set_ticks_position('right')
ax11.spines['right'].set_color('grey')  # 右侧坐标轴颜色

ax1.tick_params(axis='y', colors='k', labelcolor='k',width=0.5, direction='in', length=3)  # Y轴刻度
ax11.tick_params(axis='y', colors='grey', labelcolor='grey',width=0.5, direction='in', length=3)  # Y轴刻度
ax1.tick_params(axis='x', colors='k', labelcolor='k',width=0.5, direction='in', length=3)  # Y轴刻度
ax11.tick_params(axis='x', colors='k', labelcolor='grey',width=0.5, direction='in', length=3)  # Y轴刻度
ax11.set_yticks(range(-8,1,4))
ax11.set_xticklabels([])
ax11.set_yticklabels([8,4,0])
ax1.legend(loc=[1.02,0],frameon=False)
ax11.legend(loc=[1.02,0.1],frameon=False)
ax11.text(175,-8,f'×10$^{s}$',ha='right',va='top', color='grey')
plt.savefig(savepic,dpi=400,bbox_inches='tight')#,dpi=400
plt.show()

In [ ]:
y=hu_a_gt10
y.max()

In [ ]:
savepic=os.path.join(fig_path,'3-2 compare with PLD global grid.png')
fig=plt.figure(figsize=(6.12,5),dpi=300)#fig=plt.figure(figsize=(18,7.9))
gs = gridspec.GridSpec(2,2,width_ratios=[6,0.12],height_ratios=[1,1],wspace =0.0,hspace =0.05)

columns=['diff_a0','diff_c0']
grid_labels=[[0,0,1,0],[0,0,0,1]]
titles=['Difference of lake area (km$^2$)','Difference of lake count']

RdYlBu_r = plt.colormaps.get_cmap('RdBu_r')#
newcolors = RdYlBu_r(np.linspace(0,1, 256))
terrain_map = ListedColormap(newcolors)
    
b1=300
b2=1200
norms=[colors.Normalize(vmin=-b1, vmax=b1),colors.Normalize(vmin=-b2, vmax=b2)]
ticks=[np.arange(-b1,b1+1,100),np.arange(-b2,b2+1,400)]

## 面积、数量纬度分布图
### a.格网面积
ax22 = plt.subplot(gs[0, 1])
ax2=plt.subplot(gs[0, 0])
ax2.text(-175,75,sub_fig_name[0],fontsize=12)
for spine in ax2.spines.values():
        spine.set_visible(False)
legend_kwds={'orientation': "vertical",'extend':'both','ticks': ticks[0]}
add_background_fig(ax2,ax22,latlon_1degree_SJ_last_data,terrain_map,columns[0],legend_kwds,norms[0])
ax22.text(5,0, titles[0],rotation=90,ha='right',va='center')

### b.格网数量
ax44 = plt.subplot(gs[1, 1])
ax4=plt.subplot(gs[1, 0])
ax4.text(-175,75,sub_fig_name[1],fontsize=12)
for spine in ax4.spines.values():
        spine.set_visible(False)
legend_kwds={'orientation': "vertical",'extend':'both','ticks': ticks[1]}
add_background_fig(ax4,ax44,latlon_1degree_SJ_last_data,terrain_map,columns[1],legend_kwds,norms[1])
ax44.text(5,0, titles[1],rotation=90,ha='right',va='center')

# ax4.legend(loc=[1.02,0.1],frameon=False)
plt.savefig(savepic,dpi=400,bbox_inches='tight')#,dpi=400
plt.show()

In [ ]:
color=color_list[3:6]

savepic=os.path.join(fig_path,'3-2 compare with PLD Latitude longitude.png')
fig=plt.figure(figsize=(5,5),dpi=300)#fig=plt.figure(figsize=(18,7.9))
gs = gridspec.GridSpec(3,3,height_ratios=[1,1,2],width_ratios=[1,1,1.5],wspace =0.1,hspace =0.2)
## a.
ax1=plt.subplot(gs[0,:])
ax1.text(-175,14,sub_fig_name[0],fontsize=12)
s=4
y_top=Lon_data['hu_c0'].values/(10.0**s)
plt.tick_params(labelbottom=False,labeltop=False,labelleft=True, labelright=False)
# draw_line(ax1,[-180,180],range(-180,181,60),[0,2],[0,1,2],color_list[-1],0.8,longitude,y_top)
y=Lon_data['hu_c1'].values/(10.0**s)#GL_c0
y_bottom=y_top-y
y=Lon_data['hu_c3'].values/(10.0**s)+Lon_data['hu_c10'].values/(10.0**s)#GL_c0
y_bottom_2=y_bottom-y
ax1.bar(longitude+0.5,y_top, color=color[0],width=1,label='GLAKESPlus (0.005-0.01 km$^2$)')
ax1.bar(longitude+0.5,y_bottom, color=color[1],width=1,label='GLAKESPlus (0.01-0.1 km$^2$)')
ax1.bar(longitude+0.5,y_bottom_2, color=color[2],width=1,label='GLAKESPlus (>0.1 km$^2$)')
y=P_Lon_data['pld_c0'].values/(10.0**s)
ax1.plot(longitude+0.5, y, color=color_list[-1], linestyle = "-",linewidth = 0.4,label='PLD')

y=y-(P_Lon_data['pld_c3'].values+P_Lon_data['pld_c10'].values)/(10.0**s)
ax1.plot(longitude+0.5, y, color=red_color_list[1], linestyle = "-",linewidth = 0.4,label='PLD (0.01-0.1)')
ax1.text(-190,17,f'×10$^{s}$',ha='right',va='top', color='k')
ax1.set_ylabel('Count', color='k')
ax1.set_xlim([-180,180])
ax1.set_xticks(range(-180,181,60))
ax1.set_ylim([0,17])
ax1.set_yticks([0,5,10,15])
ax1.spines[['top', 'right']].set_visible(False)
## b.
ax2=plt.subplot(gs[1,:])
ax2.text(-175,8,sub_fig_name[1],fontsize=12)
s=4
y_top=Lon_data['hu_a0'].values/(10.0**s)
# draw_line(ax1,[-180,180],range(-180,181,60),[0,2],[0,1,2],color_list[-1],0.8,longitude,y_top)
y=Lon_data['hu_a1'].values/(10.0**s)#GL_c0
y_bottom=y_top-y
y=(Lon_data['hu_a3'].values+Lon_data['hu_a10'].values)/(10.0**s)#GL_c0
y_bottom_2=y_bottom-y
ax2.bar(longitude+0.5,y_top, color=color[0],width=1,label='GLAKESPlus (0.005-0.01 km$^2$)')
ax2.bar(longitude+0.5,y_bottom, color=color[1],width=1,label='GLAKESPlus (0.01-0.1 km$^2$)')
ax2.bar(longitude+0.5,y_bottom_2, color=color[2],width=1,label='GLAKESPlus (>0.1 km$^2$)')
y=P_Lon_data['pld_a0'].values/(10.0**s)
ax2.plot(longitude+0.5, y, color=color_list[-1], linestyle = "-",linewidth = 0.4,label='PLD')

y=y-(P_Lon_data['pld_a3'].values+P_Lon_data['pld_a10'].values)/(10.0**s)
ax2.plot(longitude+0.5, y, color=red_color_list[1], linestyle = "-",linewidth = 0.4,label='PLD (0.01-0.1)')
# y=y-P_Lon_data['pld_a3'].values/(10.0**s)-P_Lon_data['pld_a10'].values/(10.0**s)
# ax2.plot(longitude+0.5, y, color=color_list[-1], linestyle = "-",linewidth = 0.4,label='PLD (0.01-0.1)')
# ax2.legend(loc=(0.65, 0.3),frameon=False)
ax2.text(-190,10,f'×10$^{s}$',ha='right',va='top', color='k')
ax2.set_xlabel('Longitude')
ax2.xaxis.set_label_position("top")
ax2.set_ylabel('Area (km$^2$)', color='k')
ax2.set_xlim([-180,180])
ax2.set_xticks(range(-180,181,60))
ax2.set_ylim([0,10])
ax2.set_yticks([0,4,8])
ax2.spines[['top', 'right']].set_visible(False)

## c.
ax1=plt.subplot(gs[2,0])
ax1.text(0.4,75,sub_fig_name[2],fontsize=12)
s=5
y_top=Lat_data['hu_c0'].values/(10.0**s)

# draw_line(ax1,[-180,180],range(-180,181,60),[0,2],[0,1,2],color_list[-1],0.8,longitude,y_top)
y=Lat_data['hu_c1'].values/(10.0**s)#GL_c0
y_bottom=y_top-y
y=Lat_data['hu_c3'].values/(10.0**s)+Lat_data['hu_c10'].values/(10.0**s)#GL_c0
y_bottom_2=y_bottom-y
ax1.barh(latitude+0.5,y_top, color=color[0],height=1,label='GLAKESPlus (0.005-0.01 km$^2$)')
ax1.barh(latitude+0.5,y_bottom, color=color[1],height=1,label='GLAKESPlus (0.01-0.1 km$^2$)')
ax1.barh(latitude+0.5,y_bottom_2, color=color[2],height=1,label='GLAKESPlus (>0.1 km$^2$)')
y=P_Lat_data['pld_c0'].values/(10.0**s)
ax1.plot(y,latitude+0.5, color=color_list[-1], linestyle = "-",linewidth = 0.4,label='PLD')

y=y-(P_Lat_data['pld_c3'].values+P_Lat_data['pld_c10'].values)/(10.0**s)
ax1.plot( y,latitude+0.5, color=red_color_list[1], linestyle = "-",linewidth = 0.4,label='PLD (0.01-0.1)')

ax1.text(6,-55,f'×10$^{s}$',ha='right',va='bottom', color='k')
ax1.set_ylabel('Latitude', color='k')
ax1.set_xlabel('Count')
ax1.set_xlim([0,6])
ax1.set_xticks(range(0,6))
ax1.set_ylim([-60,90])
ax1.set_yticks(range(-60,91,30))
ax1.spines[['top', 'right']].set_visible(False)

## d.
ax2=plt.subplot(gs[2,1])
ax2.text(1,75,sub_fig_name[3],fontsize=12)
plt.tick_params(labelbottom=True,labeltop=False,labelleft=False, labelright=False)
s=4
y_top=Lat_data['hu_a0'].values/(10.0**s)
# draw_line(ax1,[-180,180],range(-180,181,60),[0,2],[0,1,2],color_list[-1],0.8,longitude,y_top)
y=Lat_data['hu_a1'].values/(10.0**s)#GL_c0
y_bottom=y_top-y
y=Lat_data['hu_a3'].values/(10.0**s)+Lat_data['hu_a10'].values/(10.0**s)#GL_c0
y_bottom_2=y_bottom-y
ax2.barh(latitude+0.5,y_top, color=color[0],height=1,label='GLAKESPlus (0.005-0.01 km$^2$)')
ax2.barh(latitude+0.5,y_bottom, color=color[1],height=1,label='GLAKESPlus (0.01-0.1 km$^2$)')
ax2.barh(latitude+0.5,y_bottom_2, color=color[2],height=1,label='GLAKESPlus (>0.1 km$^2$)')
y=P_Lat_data['pld_a0'].values/(10.0**s)
ax2.plot(y,latitude+0.5, color=color_list[-1], linestyle = "-",linewidth = 0.4,label='PLD')
y=y-(P_Lat_data['pld_a3'].values+P_Lat_data['pld_a10'].values)/(10.0**s)
ax2.plot(y,latitude+0.5, color=red_color_list[1], linestyle = "-",linewidth = 0.4,label='PLD (>0.1)')
ax2.legend(loc=(1, 0.3),frameon=False)
ax2.text(16,-55,f'×10$^{s}$',ha='right',va='bottom', color='k')
ax2.set_xlabel('Area (km$^2$)')
ax2.set_xlim([0,15])
ax2.set_xticks(range(0,16,4))
ax2.set_ylim([-60,90])
ax2.set_yticks(range(-60,91,30))
ax2.spines[['top', 'right']].set_visible(False)
plt.savefig(savepic,dpi=400,bbox_inches='tight')#,dpi=400
plt.show()